# 0. Imports

In [27]:
import pandas as pd
import numpy as np
import json
import multiprocessing

# 1. Get & Create Data

In [6]:
#Get & Format Data from content.csv
content = pd.read_csv('../raw_data/content.csv')
content_df = content.groupby('track_id', as_index=False).first()
content_df = content_df.reset_index(drop=True)
# content_df['track_name'] = content_df['track_name'].astype(str)
# content_df['artists'] = content_df['artists'].astype(str)
content_df = content_df[['track_id', 'track_name', 'artists']]

In [7]:
#Print Colab-Slices Locations (to check if they're correct)

slice_amount = 3
for i in range(slice_amount):
    #json_file_paths.append(f'../raw_data/playlist_data/mpd.slice.{i*1000}-{999+ i * 1000}.json')
    print((f'../raw_data/playlist_data/mpd.slice.{i*1000}-{999+ i * 1000}.json'))

../raw_data/playlist_data/mpd.slice.0-999.json
../raw_data/playlist_data/mpd.slice.1000-1999.json
../raw_data/playlist_data/mpd.slice.2000-2999.json


In [26]:
# Specify the path to your JSON file
json_file_paths = [f'../raw_data/playlist_data/mpd.slice.{i*1000}-{999+ i * 1000}.json'
                    for i in range(slice_amount)]

# Read the JSON file
json_data = []
for i in range(len(json_file_paths)):
    with open(json_file_paths[i], 'r') as file:
        json_data.append(json.load(file))

In [28]:
# Specify the path to your JSON file
json_file_paths = [f'../raw_data/playlist_data/mpd.slice.{i*1000}-{999+ i * 1000}.json'
                    for i in range(slice_amount)]

def load_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

pool = multiprocessing.Pool()
json_data = pool.map(load_json_file, json_file_paths)
pool.close()
pool.join()

In [9]:
#Get Track Sum per Batch
track_sum_per_batch = 0
batch = 0
for i in range(len(json_data[batch]['playlists'])):
    playlist_i = json_data[batch]['playlists'][i]['tracks']
    track_sum_per_batch += len(playlist_i)

In [10]:
#Get Track Sum
track_sum = 0
for i in range(len(json_data)):
    json_i = json_data[i]['playlists']
    for j in range(len(json_i)):
        playlist = json_data[i]['playlists'][j]['tracks']
        track_sum += len(playlist)

In [11]:
#Get Track List Per Batch

tracklist_per_batch = []
batch = 0

for i in range(len(json_data[batch]['playlists'])):
    playlist_i = json_data[batch]['playlists'][i]['tracks']
    for j in range(len(playlist_i)):
        tracklist_per_batch.append(f'{i}' + ': ' + playlist_i[j]['artist_name'] + ' - ' + playlist_i[j]['track_name'])

In [29]:
#Get Track List


tracklist = []
for k in range (len(json_data)):
    for i in range(len(json_data[k]['playlists'])):
        playlist_i = json_data[k]['playlists'][i]['tracks']
        for j in range(len(playlist_i)):
            tracklist.append(f'{k*1000 + i}' + ' ||| ' + playlist_i[j]['artist_name']
                             + ' ||| ' + playlist_i[j]['track_name'] + ' ||| ' + str(playlist_i[j]['track_uri']))
            
#Split Tracklist (to create DF)

split_tracklist = [item.split(' ||| ') for item in tracklist]

for i in range(len(split_tracklist)):
    split_tracklist[i][3] = split_tracklist[i][3].replace('spotify:track:', '')

In [30]:
tracklist = [f'{k*1000 + i} ||| {track["artist_name"]} ||| {track["track_name"]} ||| {track["track_uri"].replace("spotify:track:", "")}'
             for k, data in enumerate(json_data)
             for i, playlist in enumerate(data['playlists'])
             for track in playlist['tracks']]

split_tracklist = [item.split(' ||| ') for item in tracklist]
tracklist_df = pd.DataFrame(split_tracklist, columns=['playlist_number', 'artist_name', 'track_name', 'track_id'])

In [14]:
# Create DF
tracklist_df = pd.DataFrame(split_tracklist, columns=['playlist_number', 'artist_name', 'track_name', 'track_id'])

In [15]:
#Merge DFs on Track_ID (to get only the Tracks available in both Datasets)

merged_df = pd.merge(tracklist_df, content_df, on='track_id', how='inner')
clean_df = merged_df.drop(columns=['track_name_y', 'artists'])
clean_unique = clean_df.groupby('track_id', as_index=False).first()
merged_unique = clean_unique.reset_index(drop=True)

In [16]:
# #Create Dict per Track (to get all Playlists containing the Track)
# #Not operational

# trackdict = {}
# for i in range(10):
#     track = tracklist[i].split(' ||| ', tracklist[i])[1]
#     trackdict[f'{i}'] = []
#     for j in range(len(tracklist)):
#         if track in tracklist[j]:
#             trackdict[f'{i}'].append(tracklist[j].split(' ||| ')[0])
#     print(f'done with track number {i}')
# trackdict

# 2. Create Track_Matrix


In [17]:
playlist_df = clean_df.groupby('track_id', as_index=False).agg({'playlist_number': list})
playlist_df = playlist_df.reset_index(drop=True)

In [18]:
# Get all playlists containing the track

# Specify the track_id you want to locate
target_track_id = '7zkLpY72g6lKQbiHDqri1S'

# Use loc[] to locate the rows with the specific track_id
specific_rows = playlist_df.loc[playlist_df['track_id'] == target_track_id]

playlistlist = specific_rows['playlist_number'].tolist()

In [19]:
# # One Hot Encode playlists into df

# # Get all unique playlists
# unique_playlists = []
# for i in range(len(playlist_df)):
#     unique_playlists += playlist_df['playlist_number'][i]
# unique_playlists = list(set(unique_playlists))
# unique_playlists.sort()

In [22]:
unique_playlists = playlist_df['playlist_number'].explode().unique()
unique_playlists.sort()

In [23]:
# Create track_to_playlist_matrix with shape:
# (len(playlist_df) = Amount of unique Tracks, len(unique_playlists) = Amount of unique Playlists)
# Index: (playlist_df['track_id'], unique_playlists)

track_to_playlist_matrix = np.zeros((playlist_df.shape[0], len(unique_playlists)))

for i, playlists in enumerate(playlist_df['playlist_number']):
    track_to_playlist_matrix[i, np.searchsorted(unique_playlists, playlists)] = 1

In [24]:

track_matrix_2 = track_to_playlist_matrix @ track_to_playlist_matrix.T
track_matrix_2

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  2., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [32]:
# Create track_matrix with shape:
# (len(playlist_df) = Amount of unique Tracks)**2
# Index: (playlist_df['track_id'], playlist_df['track_id'])
# with track_matrix[i][j] = amount of same playlists for track i and track j

track_matrix = track_to_playlist_matrix @ track_to_playlist_matrix.T
track_matrix

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  2., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [ ]:
# give maximum value in matrix
max_value = np.max(track_matrix)
max_value

In [38]:
normalize = True
if normalize == True:
    #normalize track_matrix
    for i in range(len(track_matrix)):
        if track_matrix[i, i] != 1:
            # Normalize values
            track_matrix[i, :] /= track_matrix[i, i]
            track_matrix[:, i] /= track_matrix[i, i]
type(track_matrix)

numpy.ndarray

In [36]:
track_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
def create_colab_recommendations (track_matrix: np.ndarray, track_id):
    """
    Returns a list of recommended tracks for a given track_id
    Based on track_matrix
    (aka tracks that are in the same playlists as the given track_id)
    """
    idx = playlist_df.loc[playlist_df['track_id'] == target_track_id].index[0]
    track_matrix[idx]
    sorted_indices = np.argsort(-track_matrix[idx])
    sorted_values = track_matrix[idx][sorted_indices]
    merged_array = np.column_stack((sorted_indices, sorted_values))
    return merged_array

In [ ]:
def create_colab_recommendations(track_matrix: np.ndarray, target_track_id):
    """
    Returns a list of recommended tracks for a given track_id
    Based on track_matrix
    (aka tracks that are in the same playlists as the given track_id)
    """
    idx = np.where(playlist_df['track_id'] == target_track_id)[0][0]
    sorted_indices = np.argsort(-track_matrix[idx])
    merged_array = np.column_stack((sorted_indices, track_matrix[idx][sorted_indices]))
    return merged_array

hier könnten wir die daten normalisieren, also alle werte auf 0-1 bringen   
wenn ein track in vielen verschiedenen playlists landet   
(aktuelles maximum: 139x) könnte eine cooccurence weniger aussagekräftig sein   
dazu: jede cooccurence teilen   
durch: Wert auf der Diagonalen (also wie oft ein track in der playlist vorkommt)   
dann weighting Wert Alpha als Variable einbauen   